In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import sys
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
ROOT_PATH = Path().resolve().parent.parent
if str(ROOT_PATH) not in sys.path:
    sys.path.insert(1, str(ROOT_PATH))
DIR_PATH = Path().resolve()
from frequency_response import FrequencyResponse
from research.peq_optimizer.neural_peq import NeuralPEQ

## Create Data

In [ ]:
# Read equalization curves from results as data
# TODO: Create data generator for unlimited data
data_path =  DIR_PATH.joinpath('data.npy')
if data_path.is_file():
    # Saved previously to numpy file
    data = np.load(data_path)
else:
    # Not saved, read frequency response CSVs
    data = []
    for fp in ROOT_PATH.joinpath('results').glob('**/*.csv'):
        fr = FrequencyResponse.read_from_csv(fp)
        fr.interpolate()
        data.append(fr.equalization)
    data = np.vstack(data)
    np.save(data_path, data)

# Split data into training and testing
n_train = int(len(data) * 0.8)
n_test = len(data) - n_train
train = data[:n_train]
test = data[n_train:]

In [ ]:
import optuna

def objective(trial):
    #hidden_size = trial.suggest_int('hidden_size', 512, 2048, log=True)
    #dropout = trial.suggest_float('dropout', 0.0, 0.9)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 6e-4, log=True)
    learning_rate_decay = trial.suggest_float('learning_rate_decay', 0.06, 0.09)
    #gain_normalization = trial.suggest_float('gain_normalization', 1.0, 10.0)
    warmup_epochs = trial.suggest_int('warmup_epochs', 3, 10)
    peq = NeuralPEQ(
        n_filters=10,
        hidden_size=2000,
        dropout=0.0,
        learning_rate=learning_rate,
        learning_rate_decay=learning_rate_decay,
        log_f=True,
        q_normalization=1.0,
        gain_normalization=1.0,
        warmup_epochs=warmup_epochs)
    try:
        history = peq.train(train)
    except:
        return 999.0
    loss = np.array(history.history['val_loss'])
    loss = loss[np.logical_not(np.isnan(loss))]
    loss = np.min(loss)
    return loss

study = optuna.create_study()
study.optimize(objective, n_trials=20)

study.best_params  # E.g. {'x': 2.002108042}

In [ ]:
peq = NeuralPEQ(
    n_filters=10,
    hidden_size=2000,
    dropout=0.0,
    learning_rate=4.24e-4,
    learning_rate_decay=0.074,
    log_f=True,
    q_normalization=1.0,
    gain_normalization=1.0,
    warmup_epochs=8)
history = peq.train(train)

In [ ]:
for i in range(3,4):
    filters = peq.infer(test[i:i+1])
    fr = FrequencyResponse(name='test', raw=test[i], equalization=np.squeeze(peq.fr(filters)), target=test[i])
    fr.error = fr.raw - fr.target
    fig, ax = fr.plot_graph(show=False, raw=True, target=False, error=False)
    for i, filt in enumerate(filters[0]):
        ax.plot(fr.frequency, np.squeeze(peq.fr(np.expand_dims(np.expand_dims(filt, axis=0), axis=0))), label=f'{filt[0]:.2f} Hz, {filt[1]:.2f} Q, {filt[2]:.2f} dB')
    ax.legend()
    plt.show()

In [ ]:
p = np.array([
    [ 2.8177048e+01, 4.6592656e-01, 3.6886775e+00],
    [ 7.5532819e+02, 6.3449883e-01, 4.0352540e+00],
    [ 2.1447576e+03, 4.8449698e-01, -7.1967192e+00],
    [ 3.0843210e+03, 2.0635529e+00, 6.9372535e+00],
    [ 6.6201890e+03, 8.5014981e-01, 6.9353619e+00],
    [ 1.9410781e+04, 2.7364632e-01, -2.0113779e+01]], dtype='float32')
p = np.expand_dims(p, axis=0)
raw = peq.fr(p)
fr = FrequencyResponse(name='peq', raw=test[0], target=np.squeeze(raw))
fr.error = fr.raw - fr.target
fr.plot_graph()
plt.show()

In [ ]:
d = ((p[:, :, 1] - 1.41) ** 2).T
d = np.hstack([np.ones((p.shape[1], p.shape[2]-1)), d])

In [ ]:
d

In [ ]:
p

In [ ]:
p*d

In [ ]:
x = np.array([13.655136108398438, 16.40733528137207, 18.72626495361328, 19.614147186279297, 19.742990493774414, 19.676668167114258, 19.632984161376953])

In [ ]:
x[np.logical_not(np.isnan(x))]